## PopCount8

In this tutorial, we show how to construct a circuit to compute an 8-bit PopCount (population count).

In [1]:
import magma as m
m.set_mantle_target("ice40")

In this example, we are going to use the built-in `fulladder` from `Mantle`.

In [2]:
from mantle import fulladder

import lattice ice40
import lattice mantle40


A common name for a full adder is a carry-sum adder, or `csa`.
Let's define two csa functions to help us construct the popcount.

In [3]:
# 2 input 
def csa2(I0, I1):
    return m.bits(fulladder(I0, I1, 0))

# 3 input
def csa3(I0, I1, I2):
    return m.bits(fulladder(I0, I1, I2))

To construct the 8-bit popcount, we first use 3 csa's to sum
bits 0 through 2, 3 through 5, and 6 through 7.
This forms 3 2-bit results.
We can consider the results to be two columns, one for each *place*.
The first column is the 1s and the second column is the 2s.

We then use two fulladders to sum these columns.
We continue summing 3-bits at a time until we get a single bit in each column.

A common way to show these operations is with *Dadda dot notation*
which shows how many bits are in each colum.

In [4]:
def popcount8(I):
    # Dadda dot notation (of the result)
    # o o     csa0_0_21 - row 0, bits 2 and 1
    # o o     csa0_1_21 - row 1, bits 2 and 1
    # o o     csa0_2_21 - row 2, bits 2 and 1
    csa0_0_21 = csa3(I[0], I[1], I[2])
    csa0_1_21 = csa3(I[3], I[4], I[5])
    csa0_2_21 = csa2(I[6], I[7])

    #   o o   csa1_0_21 - row 0, bits 2 and 1
    # o o     csa1_1_43 - row 1, bits 4 and 2
    csa1_0_21 = csa3(csa0_0_21[0], csa0_1_21[0], csa0_2_21[0])
    csa1_1_42 = csa3(csa0_0_21[1], csa0_1_21[1], csa0_2_21[1])

    # o o     csa2_0_42 - row 0, bits 4 and 2
    csa2_0_42 = csa2(csa1_0_21[1], csa1_1_42[0])

    # o o     csa3_0_84 - row 0, bits 8 and 4 
    csa3_0_84 = csa2(csa1_1_42[1], csa2_0_42[1])
    
    return m.bits([csa1_0_21[0], csa2_0_42[0], csa3_0_84[0], csa3_0_84[1]])

## Test bench

In order to test the popcount circuit,
we setup the IceStick board
to have eight inputs and four outputs.
As before, `J1` will be used for inputs and `J3` for outputs.

In [5]:
from loam.boards.icestick import IceStick

icestick = IceStick()
for i in range(8):
    icestick.J1[i].input().on()
for i in range(4):
    icestick.J3[i].output().on()
    
main = icestick.DefineMain()

m.wire( popcount8(main.J1), main.J3 )

m.EndDefine()

In [6]:
m.compile('build/popcount8', main)

And use our `yosys`, `arcachne-pnr`, and `icestorm` tool flow.

In [7]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif popcount8.blif' popcount8.v
arachne-pnr -q -d 1k -o popcount8.txt -p popcount8.pcf popcount8.blif 
icepack popcount8.txt popcount8.bin
iceprog popcount8.bin

init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x64 0x34 0x65 0x03 0x00 0x71 0x00 0x26 0x27 0x12 0x16 0xD3 0xE4
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.


You can test the program by connecting up some switches and LEDs to the headers. You should see the count of the inputs displayed on the LEDs.

There is a more general version of `PopCount` in the `Mantle` library `util.compressor`.